In [ ]:
import os
import numpy as np
import keras
from keras.engine import  Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import InceptionV3
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau

In [ ]:
from keras import backend as k
k.clear_session()

In [ ]:
print(keras.__version__)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Data Preparation

In [ ]:
# data are in dataset folder in zipped format
!ls "drive/My Drive/ENGR635-Deep Learning System Design Project/Dataset/fer2013/"

In [ ]:
!ls 

In [ ]:
# This creates a temporary folder in drive root folder, so it will have to be reloaded again when required after terminating the session
# Permanent data are stored in dataset in zipped format
# This copies the zipped file and store in root of google drive temporarily 
# This allows colab to run faster by taking data from root folder, rather than from actual drive location.
! rm -rf Training; mkdir Training
! unzip -q "drive/My Drive/ENGR635-Deep Learning System Design Project/Dataset/fer2013/Training.zip" -d Training

! rm -rf Validation; mkdir Validation
! unzip -q "drive/My Drive/ENGR635-Deep Learning System Design Project/Dataset/fer2013/PublicTest.zip" -d Validation

! rm -rf Test; mkdir Test
! unzip -q "drive/My Drive/ENGR635-Deep Learning System Design Project/Dataset/fer2013/PrivateTest.zip" -d Test

In [ ]:
!ls

In [ ]:
!ls -l Training/

In [ ]:
# Counting the images in each training data categories
%%bash
root='Training/'
IFS=$(echo -en "\n\b")
(for dir in $(ls -1 "$root")
    do printf "$dir: " && ls -i "$root$dir" | wc -l
 done)

In [ ]:
%%bash
root='Validation/'
IFS=$(echo -en "\n\b")
(for dir in $(ls -1 "$root")
    do printf "$dir: " && ls -i "$root$dir" | wc -l
 done)

In [ ]:
%%bash
root='Test/'
IFS=$(echo -en "\n\b")
(for dir in $(ls -1 "$root")
    do printf "$dir: " && ls -i "$root$dir" | wc -l
 done)

In [ ]:
# Setting up path for training, validation and test directory
train_dir = "Training/"
validation_dir = "Validation/"
test_dir = "Test/"

## All data are ready

### Setting up Image generator with data augmentation

In [ ]:
# Setting image height and width
img_height = 224
img_width = 224

In [ ]:
# Image Data Generator setup
batch_size = 50
train_datagen = ImageDataGenerator(rescale=1./255,
                                   featurewise_center=False,
                                   featurewise_std_normalization=False,
                                   rotation_range=30,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=0.1,
                                   horizontal_flip=True
                                   )
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    shuffle = True,
    class_mode='categorical'
)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'categorical'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'categorical'
)

In [ ]:
print(train_generator.class_indices)
print(validation_generator.class_indices)
print(test_generator.class_indices)

In [ ]:
def class_weight_computer():
  """
    Training Data categories and number of samples in them
    Angry: 3995
    Disgust: 436
    Fear: 4097
    Happy: 7215
    Neutral: 4965
    Sad: 4830
    Surprise: 3171
  """
  samples_per_label = [3995, 436, 4097, 7215, 4965, 4830, 3171]
  total_samples = sum(samples_per_label)
  return dict([(i, total_samples/(7*j)) for (i,j) in enumerate(samples_per_label)]) # https://scikit-learn.org/stable/modules/generated/sklearn.utils.class_weight.compute_class_weight.html
class_weights = class_weight_computer()

### Loading pretrained model

In [ ]:
conv_base = InceptionV3(weights="imagenet",
                        include_top=False,
                        input_shape=(img_height, img_width, 3)
                        )

In [ ]:
conv_base.summary()

In [ ]:
print("Number of trainable weights before freezing the conv base:", len(conv_base.trainable_weights))
conv_base.trainable = False
print("Number of trainable weights after freezing the conv base:", len(conv_base.trainable_weights))

In [ ]:
conv_base.summary()

### Adding Classifier on top of pretrained network

In [ ]:
from keras.layers import GlobalAveragePooling2D
DROP_OUT_RATE = 0.5
last_layer = conv_base.output

x = GlobalAveragePooling2D(name='global_avg_pool_1')(last_layer)
x = Dropout(DROP_OUT_RATE)(x)
x = Dense(1024, activation='relu', name='fc1')(x)
x = Dropout(DROP_OUT_RATE)(x)
x = Dense(1024, activation='relu', name='fc2')(x)
out = Dense(7, activation='softmax', name='classifier')(x)

model = Model(conv_base.input, out)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.0001)
model.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# Model Training
EPOCHS = 50
training_samples = 28709
validation_samples = 3589
test_samples = 3589
history = model.fit_generator(
    train_generator,
    steps_per_epoch=training_samples//batch_size,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_samples//batch_size,
    shuffle=True,
    class_weight=class_weights,
    callbacks=[reduce_lr]
)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
loss = history.history['loss']

val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'r', label="Training accuracy")
plt.plot(epochs, val_acc, 'b', label="Validation accuracy")
plt.title('Training and Validation Accuracy with freezed Conv base of InceptionV3')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'r', label="Training Loss")
plt.plot(epochs, val_loss, 'b', label="Validation Loss")
plt.title('Training and Validation Loss with freeze Conv base of InceptionV3')
plt.legend()
plt.show()

In [ ]:
print('\nEvaluate on Validation data')
results_validation = model.evaluate_generator(validation_generator, validation_samples//batch_size)
print('Validation loss, Validation Accuracy:', results_validation)

In [ ]:
epoch_str = '-EPOCHS_' + str(EPOCHS)
val_acc = 'val_acc_%.3f' % results_validation[1]

In [ ]:
print(epoch_str)
print(val_acc)

In [ ]:
# Saving the model
model_path = "drive/My Drive/ENGR635-Deep Learning System Design Project/Models/InceptionV3/InceptionV3" + epoch_str + val_acc + '.h5'
model.save(model_path)

### Finetune

### Fine tuning the last inception block

Image generators and data preparation codes are pre-loaded before the below session

In [ ]:
from keras.models import load_model
model = load_model("drive/My Drive/ENGR635-Deep Learning System Design Project/Models/InceptionV3/InceptionV3-EPOCHS_50val_acc_0.388.h5")

In [ ]:
model.summary()

In [ ]:
# Unfreezing all the layers
for layer in model.layers:
  layer.trainable = True

In [ ]:
model.summary()

In [ ]:
# For fine tuning, selecting the last Inception Block training.
set_trainable = False
for layer in model.layers:
  if layer.name == "mixed9":
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.00001)
# default learning rate was 0.001, so now reduced it by 10
model.compile(optimizer=optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
EPOCHS = 50
training_samples = 28709
validation_samples = 3589
test_samples = 3589
history = model.fit_generator(
    train_generator,
    steps_per_epoch=training_samples//batch_size,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_samples//batch_size,
    shuffle=True,
    class_weight=class_weights,
    callbacks=[reduce_lr]
)

In [ ]:
print('\nEvaluate on Validation data')
results_validation = model.evaluate_generator(validation_generator, validation_samples//batch_size)
print('Validation loss, Validation Accuracy:', results_validation)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
loss = history.history['loss']

val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'r', label="Training accuracy")
plt.plot(epochs, val_acc, 'b', label="Validation accuracy")
plt.title('Training and Validation Accuracy with fine tuning of InceptionV3')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'r', label="Training Loss")
plt.plot(epochs, val_loss, 'b', label="Validation Loss")
plt.title('Training and Validation Loss with fine tuning of InceptionV3')
plt.legend()
plt.show()

In [ ]:
epoch_str = '-EPOCHS_' + str(EPOCHS)
val_acc = 'test_acc_%.3f' % results_validation[1]
print(val_acc)
print(epoch_str)

In [ ]:
# Saving the model
model.save("drive/My Drive/ENGR635-Deep Learning System Design Project/Models/InceptionV3/" + 'InceptionV3_finetuned' + epoch_str + val_acc + '.h5')

---


### Fine tuning the last two inception blocks

Image generators and data preparation codes are pre-loaded before the below session

In [ ]:
from keras.models import load_model
model = load_model("drive/My Drive/ENGR635-Deep Learning System Design Project/Models/InceptionV3/InceptionV3_finetuned-EPOCHS_50val_acc_0.590.h5")

In [ ]:
# Unfreezing all the layers
for layer in model.layers:
  layer.trainable = True

In [ ]:
model.summary()

In [ ]:
# For fine tuning, selecting the 2 last Inception Block training.
set_trainable = False
for layer in model.layers:
  if layer.name == "mixed8":
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.000001)
# default learning rate was 0.001, which was reduced to 0.0001 during first fine tuning now further reducing to 0.00001
model.compile(optimizer=optimizers.Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# Model training
EPOCHS = 50
training_samples = 28709
validation_samples = 3589
test_samples = 3589
history = model.fit_generator(
    train_generator,
    steps_per_epoch=training_samples//batch_size,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_samples//batch_size,
    shuffle=True,
    class_weight=class_weights,
    callbacks=[reduce_lr]
)

In [ ]:
print('\nEvaluate on Validation data')
results_validation = model.evaluate_generator(validation_generator, validation_samples//batch_size)
print('Validation loss, Validation Accuracy:', results_validation)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
loss = history.history['loss']

val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'r', label="Training accuracy")
plt.plot(epochs, val_acc, 'b', label="Validation accuracy")
plt.title('Training and Validation Accuracy after second fine tuning of InceptionV3')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'r', label="Training Loss")
plt.plot(epochs, val_loss, 'b', label="Validation Loss")
plt.title('Training and Validation Loss after second fine tuning of InceptionV3')
plt.legend()
plt.show()

In [ ]:
epoch_str = '-EPOCHS_' + str(EPOCHS)
val_acc = 'val_acc_%.3f' % results_validation[1]

In [ ]:
print(epoch_str)
print(val_acc)

In [ ]:
# Saving the model
model.save("drive/My Drive/ENGR635-Deep Learning System Design Project/Models/InceptionV3/" + 'InceptionV3_finetuned_' + epoch_str + val_acc + '.h5')

In [ ]:
print('\nEvaluate on test data')
results_test = model.evaluate_generator(test_generator, 3589//50)
print('test loss, test acc:', results_test)